<a href="https://colab.research.google.com/github/Ginko2501/SURP2022/blob/main/ATF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import csv
import math
import numpy as np

In [4]:
# variables
nodes = np.empty(shape=[0,2])
nodal_rays = np.empty(shape=[0,2])
edges = np.empty(shape=[0,2])

In [5]:
def init_polydisk (x):
  # initialize polydisk P(1,x)
  global n
  global nodes
  global nodal_rays
  global edges

  n = 4

  # init nodes
  nodes = np.empty(shape=[0,2])
  nodes = np.append(nodes, [[0,0]], axis=0)
  nodes = np.append(nodes, [[0,1]], axis=0)
  nodes = np.append(nodes, [[x,1]], axis=0)
  nodes = np.append(nodes, [[x,0]], axis=0)

  # init nodal rays
  nodal_rays = np.empty(shape=[0,2])
  nodal_rays = np.append(nodal_rays, [[1,1]], axis=0)
  nodal_rays = np.append(nodal_rays, [[1,-1]], axis=0)
  nodal_rays = np.append(nodal_rays, [[-1,-1]], axis=0)
  nodal_rays = np.append(nodal_rays, [[-1,1]], axis=0)

  # init edges
  edges = np.empty(shape=[0,2])
  edges = np.append(edges, [[0,1]], axis=0)
  edges = np.append(edges, [[x,0]], axis=0)
  edges = np.append(edges, [[0,-1]], axis=0)
  edges = np.append(edges, [[-x,0]], axis=0)
  

In [6]:
# test init_polydisk
init_polydisk(1.3)
print(nodes)

[[0.  0. ]
 [0.  1. ]
 [1.3 1. ]
 [1.3 0. ]]


In [7]:
def dist (x,y):
  return np.linalg.norm(x-y)

In [15]:
def intersect_one (i,j):
  # solve the intersection between i-th nodal ray and j-th edge
  global n
  global nodes
  global nodal_rays
  global edges

  # copy as local variables
  n1 = nodes[i]
  n2 = nodes[j]
  n3 = nodes[(j+1)%n]
  v1 = nodal_rays[i]
  v2 = edges[j]

  # solve for the intersection point
  vec = np.array([ v1[1]*n1[0]-v1[0]*n1[1], v2[1]*n2[0]-v2[0]*n2[1] ])
  mat = np.array([ [ v1[1], -v1[0] ],
            [ v2[1], -v2[0] ] ])
  itx = np.linalg.solve(mat, vec)

  # check the intersection is on the edge
  if (n2[0]==n3[0]):
    lmbda = (itx[1]-n3[1]) / (n2[1]-n3[1])
  else:
    lmbda = (itx[0]-n3[0]) / (n2[0]-n3[0])
  if (lmbda<0 or lmbda>1):
    return -1
  
  # return the rescale size
  ratio = (itx[0]-n1[0]) / v1[0]
  if (ratio<0):
    return -1
  else:
    return ratio

In [13]:
# test intersect_one
init_polydisk(2)
intersect_one(0,1)

(1.0, array([1., 1.]))

In [14]:
def intersect_all (x):
  # solve the intersecting edge by the x-th nodal ray
  global n
  global nodes
  global nodal_rays
  global edges

  # the variables for the intersecting edge
  min_edge = x
  min_ratio = math.inf
  
  for i in range(n):
    if (i==x or i==(x-1)%n):
      continue

    ratio = intersect_one(x,i)
    if (ratio==-1):
      continue
    
    if (ratio < min_ratio):
      min_edge = i
      min_ratio = ratio

  return (min_edge, nodes[x]+min_ratio*nodal_rays[x])

In [17]:
# test intersect_all
init_polydisk(0.5)
intersect_all(1)

(2, array([0.5, 0.5]))

In [ ]:
def affine_length (v):
  

In [46]:
def solve_matrix (v1, v2, w1, w2):
  # solve the eigen-direction
  vec = np.concatenate((v2,w2))
  mat = np.array([ [v1[0], v1[1], 0,   0  ],
            [0,   0,   v1[0], v1[1]], 
            [w1[0], w1[1], 0,  0,  ],
            [0,   0,   w1[0], w1[1]], ])
  res = np.linalg.solve(mat,vec)
  return np.array([ [res[0],res[1]], 
            [res[2],res[3]] ])

In [ ]:
# test solve_matrix
v1 = [-1,1]
v2 = [-1,1]
w1 = [0,1]
w2 = [1,0]
solve_matrix(v1,v2,w1,w2)

array([[ 2.,  1.],
       [-1.,  0.]])

In [ ]:
def mutate_clockwise (l, r):
  # mutate with nodal_ray < intersecting edge
  global n
  global nodes
  global nodal_rays
  global edges

  mat = solve_matrix(l,l, edges[l], edges[(l-1)%n])

In [21]:
def mutate_counterclockwise (x):
  # mutate with nodal_ray > intersecting edge
  return x

In [ ]:
def mutate (x):
  # mutate once by x-th nodal_ray
  global n
  global nodes
  global nodal_rays
  global edges

  (y, itx) = intersect_all(x)

  if (x<y):
    return mutate_clockwise(x,y)
  else:
    return mutate_counterclockwise(x,y)